In [1]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from sklearn import linear_model
import pickle
import random
from sklearn import preprocessing
import os
#import seaborn as sns

In [ ]:
import csv
file = open("data/train.csv", "r")
#file = open("mysample.csv", "r")
dl = list(csv.reader(file, delimiter=","))
file.close()

dd = {}
total_nodes = [i[0] for i in dl]

#converting to dictionary
for i  , e in enumerate(dl):
    dd[e[0]] = e[1:]


In [ ]:
len(dl[8])

In [ ]:
#uncomment for loading presaved graph
#G = pickle.load(open('data/samplegraph.pickle', 'rb'))

# comment when load is uncommented
G = nx.DiGraph()

#dl = datalist

for i in range(len(dl)):
    if(len(dl[i])==1):
        G.add_node(dl[i][0])
        #add empty node
        continue;

    for j in range(1,len(dl[i])):
      G.add_edge(dl[i][0],dl[i][j])


#nx.draw(G, with_labels=True)
#out_degree['11']


In [ ]:
#pickle.dump(G, open('samplegraph.pickle', 'wb'))
#nx.draw(G, with_labels=True)
print(G)

## Features

In [ ]:
def inedges(node):
    inedges = G.in_edges(node)
    inedges = list(inedges)
    inedges_list=[]
    for i in range (0,len(inedges)):
        inedges_list.append(inedges[i][0])

    return inedges_list

def outedges(node):
    out = G.out_edges(node)
    out = list(out)
    out_list=[]

    for i in range (0,len(out)):
        out_list.append(out[i][1])

    return out_list



def friends(source , dest):
    l1 = outedges(source)
    l2 = outedges(dest)
    if( (len(l1) == 0) or (len(l2) == 0) ):
        return 0
    flag=0

    for i in l1:
        if (i == dest):
            flag= flag+1
    #intersection = set(l1).intersection(l2)

    for i in l2:
        if(i == source):
            flag= flag+1

    if(flag==2):
        return 1

    return 0





## Dependent Features

In [ ]:
def feature_jaccard_out_follow(source , dest):

    l1 = outedges(source)
    l2 = outedges(dest)
    if( (len(l1) == 0) or (len(l2) == 0) ):
        return 0


    intersection = set(l1).intersection(l2)
    union = set(l1).union(l2)

    if(len(union)==0):
        return 0

    return len(intersection)/len(union)

def feature_jaccard_out_follower(source , dest):

    l1 = inedges(source)
    l2 = inedges(dest)
    if( (len(l1) == 0) or (len(l2) == 0) ):
        return 0


    intersection = set(l1).intersection(l2)
    union = set(l1).union(l2)

    if(len(union)==0):
        return 0

    return len(intersection)/len(union)




def feature_transitive(source , dest):
    l1 = outedges(source)
    l2 = inedges(dest)
    if( (len(l1) == 0) or (len(l2) == 0) ):
        return 0


    intersection = set(l1).intersection(l2)
    return len(intersection)

def feature_preferential(source , dest):
    #rich become richer
    return (G.in_degree(source) * G.in_degree(dest))


def feature_commonfriends_follow(source , dest):
    a= outedges(source)
    b = outedges(dest)
    intersection = set(a).intersection(b)

    return len(intersection)

def feature_commonfriends_follower(source , dest):
    a= inedges(source)
    b = inedges(dest)
    intersection = set(a).intersection(b)

    return len(intersection)


def feature_commonfriends2(source , dest):

    source_friends = []
    dest_friends = []

    for i in outedges(source):
        if(friends(i,source)):
            source_friends.append(i)


    for i in outedges(dest):
        if(friends(i,dest)):
            dest_friends.append(i)

    intersection = set(source_friends).intersection(dest_friends)
    #print(intersection)
    return len(intersection)


def feature_celebrity(source , dest):
    #a=  len(inedges(source))/len(outedges(dest))
    ine =  len(inedges(dest))
    oute = len(outedges(dest))
    if(oute==0):
        return 0
    b = ine/oute

    # considering b being celebrity , b>>>>a

    return b

def feature_botfollow(source , dest):
    #a = len(outedges(source))/len(inedges(dest))
    ine =  len(inedges(source))
    oute = len(outedges(source))

    if(ine==0):
        return 0
    b = oute/ine

    return b


def feature_cosine_for_followers(source,dest):

    l1 = inedges(source)
    l2 = inedges(dest)
    if( (len(l1) == 0) or (len(l2) == 0) ):
        return 0


    intersection = set(l1).intersection(l2)
    div = feature_preferential(source , dest)
    if(div==0):
        return 0

    return len(intersection)/div


def feature_cosine_for_followees(source,dest):
    l1 = outedges(source)
    l2 = outedges(dest)
    if( (len(l1) == 0) or (len(l2) == 0) ):
        return 0


    intersection = set(l1).intersection(l2)
    div = feature_preferential(source , dest)
    if(div==0):
        return 0

    return len(intersection)/div



def feature_compute_shortest_path_length(a,b):
    train_graph = G
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1


def feature_follows_back(a,b):
    if G.has_edge(b,a):
        return 1
    else:
        return 0

def feature_adar(a,b):
    sum=0
    try:
        n=list(set(G.successors(a)).intersection(set(G.successors(b))))
        if len(n)!=0:
            for i in n:
                l = len(list(G.predecessors(i)))
                if(l<=1):
                    return 0
                sum=sum+(1/np.log10(  l ))
            return sum
        else:
            return 0
    except:
        return 0

In [ ]:
pr = nx.pagerank(G, alpha=0.85)
#filehandler = open("hellopage", 'wb')
#pickle.dump(pr, filehandler)

In [ ]:
## katz

# if not os.path.isfile('katz.p'):
#     katz = nx.katz.katz_centrality(G,alpha=0.005,beta=1)
#     pickle.dump(katz,open('katz.p','wb'))
# else:
#     katz = pickle.load(open('katz.p','rb'))


In [ ]:
#storing pr

# with open('dictpagerank.csv', 'w') as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in pr.items():
#        writer.writerow([key, value])

In [ ]:
#reading back.
# with open('data/dictpagerank.csv') as csv_file:
#     reader = csv.reader(csv_file)
#     pr = dict(reader)



In [ ]:
# #print(outedges('1'))
# feature_transitive('1','2')
# feature_jaccard_out('1','2')
# print(feature_preferential('1','2'))
# feature_commonfriends('1','5')


In [ ]:
# testing

# def feature_commonfriends(source , dest):
#     counter = 0
#     source_friends = []
#     dest_friends = []

#     for i in outedges(source):
#         counter = counter+1;
#         print(i," outedges of source" , counter)
#         if(friends(i,source)):
#             source_friends.append(i)


#     counter=0
#     for i in outedges(dest):
#         counter = counter+1;
#         print(i," outedges of desti",counter)
#         if(friends(i,dest)):
#             dest_friends.append(i)

#     intersection = set(source_friends).intersection(dest_friends)
#     #print(intersection)
#     return len(intersection)

# feature_commonfriends('4823249' , '1941668')

## Create Dataframe

In [ ]:

#feature_list = ['from','to','feature_jaccard_out','output']
col = ['feature_jaccard_out_follow','feature_jaccard_out_follower' ,'feature_transitive','feature_preferential','feature_commonfriends_follow',
                   'feature_commonfriends_follower','feature_celebrity', 'feature_cosine_for_followers','feature_cosine_for_followees' ,
                   'feature_botfollow','pagerank_follow','pagerank_follower','feature_compute_shortest_path_length','feature_adar','output']


feature_list = col.copy()
feature_list.remove('output')

#feature_list = ['from','to','feature_jaccard_out','output']
#feature_list = ['from','to','feature_jaccard_out','feature_transitive','feature_preferential','feature_commonfriends','feature_celebrity',
#                    'feature_botfollow','output']
print(feature_list)

['feature_jaccard_out_follow', 'feature_jaccard_out_follower', 'feature_transitive', 'feature_preferential', 'feature_commonfriends_follow', 'feature_commonfriends_follower', 'feature_celebrity', 'feature_cosine_for_followers', 'feature_cosine_for_followees', 'feature_botfollow', 'pagerank_follow', 'pagerank_follower', 'feature_compute_shortest_path_length', 'feature_adar']


In [ ]:
df = pd.DataFrame(columns=col)
df


,feature_jaccard_out_follow,feature_jaccard_out_follower,feature_transitive,feature_preferential,feature_commonfriends_follow,feature_commonfriends_follower,feature_celebrity,feature_cosine_for_followers,feature_cosine_for_followees,feature_botfollow,pagerank_follow,pagerank_follower,feature_compute_shortest_path_length,feature_adar,output


In [ ]:
#df = df[0:0]
#len(df.index)

In [ ]:
### Task 2 : Make Samples
#df = df[0:0]
# positive Samples.
n = 500000

for i in range(0,n):
    #r1 = random.randint(0, len(total_nodes)) when sample size is less n
    #fn = from node
    #tn = to node
    print(i)
    r1 = random.randint(0, len(total_nodes)-1)
    fn = total_nodes[r1]
    fn_len = len(dd[fn])

    if (len(dd[fn]) == 0) :
        continue


    r2 = random.randint(0, fn_len-1)

    tn =  dd[fn][r2]



    # col = ['feature_jaccard_out_follow','feature_jaccard_out_follower' ,'feature_transitive','feature_preferential','feature_commonfriends_follow',
    #                'feature_commonfriends_follower','feature_celebrity', 'feature_cosine_for_followers','feature_cosine_for_followees' ,
    #                'feature_botfollow','pagerank_follow','pagerank_follower','feature_botfollow','compute_shortest_path_length','output']


    df.loc[len(df.index)] = [feature_jaccard_out_follow(fn,tn) ,feature_jaccard_out_follower(fn,tn), feature_transitive(fn,tn),
                            feature_preferential(fn,tn), feature_commonfriends_follow(fn,tn),feature_commonfriends_follower(fn,tn),

                            feature_celebrity(fn,tn),feature_cosine_for_followers(fn,tn),feature_cosine_for_followees(fn,tn)
                             ,feature_botfollow(fn,tn),pr[tn] ,pr[fn],feature_compute_shortest_path_length(fn,tn),feature_adar(fn,tn),1
                             ]



    # df.loc[len(df.index)] = [fn, tn,feature_jaccard_out(fn,tn) , feature_transitive(fn,tn),
    #                         feature_preferential(fn,tn), feature_commonfriends(fn,tn),
    #                         feature_celebrity(fn,tn),feature_botfollow(fn,tn),1
    #                          ]


   # df.loc[len(df.index)] = [fn, tn,feature_jaccard_out(fn,tn),1
      #                       ]

df.to_csv('biohazard.csv', index=False)


In [ ]:
#df.to_csv('biohazard.csv', index=False)
df
df.describe()


## Negative Edge Sampling

In [ ]:

for i in range(0,n):
    print(i)
    r1 = random.randint(0, len(total_nodes)-1)

    #take  node from total node list in sequence , and random node from total ndoe lsit

    #to_node.append(total_nodes[r1])
    #r1 = random.randint(0, len(total_nodes)-1)
    r2 = random.randint(0, len(total_nodes)-1)

    fn = total_nodes[r1]
    if (len(dd[fn]) == 0) :
        continue

    #to_t =  total_nodes[r1]
    #print(r1)
    #n = nx.non_neighbors(G,fn)
    #tn = next(n)
    tn =  total_nodes[r2]
    #print(i, " fn " , fn , " " ,tn)
    if (len(dd[tn]) == 0):
        continue
        #to_t_list = []

    # col = ['feature_jaccard_out_follow','feature_jaccard_out_follower' ,'feature_transitive','feature_preferential','feature_commonfriends_follow',
    #                'feature_commonfriends_follower','feature_celebrity', 'feature_cosine_for_followers','feature_cosine_for_followees' ,
    #                'feature_botfollow','pagerank_follow','pagerank_follower','feature_botfollow','compute_shortest_path_length','output']


    df.loc[len(df.index)] = [feature_jaccard_out_follow(fn,tn) ,feature_jaccard_out_follower(fn,tn), feature_transitive(fn,tn),
                            feature_preferential(fn,tn), feature_commonfriends_follow(fn,tn),feature_commonfriends_follower(fn,tn),

                            feature_celebrity(fn,tn),feature_cosine_for_followers(fn,tn),feature_cosine_for_followees(fn,tn)
                             ,feature_botfollow(fn,tn),pr[tn] ,pr[fn],feature_compute_shortest_path_length(fn,tn),feature_adar(fn,tn),0
                             ]


    #df.loc[len(df.index)] = [feature_jaccard_out(fn,tn),0
     #                        ]

df.to_csv('biohazard.csv', index=False)


In [ ]:
## comment this for if starting new  sampling search
#df = pd.read_csv('biohazard.csv')
#df['pagerank_follower'] = df['pagerank_follower'].astype(float)
#df['pagerank_follow'] = df['pagerank_follow'].astype(float)

In [ ]:
df[feature_list] = df[feature_list].astype(float)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[feature_list] = scaler.fit_transform(df[feature_list])

# standard_scaler = StandardScaler()
# X_train_standard = standard_scaler.fit_transform(X_train)
# X_test_standard = standard_scaler.transform(X_test)

## Algorithm

In [ ]:



X = df[feature_list]
y = df['output']



In [ ]:
X.describe()

,feature_jaccard_out_follow,feature_jaccard_out_follower,feature_transitive,feature_preferential,feature_commonfriends_follow,feature_commonfriends_follower,feature_celebrity,feature_cosine_for_followers,feature_cosine_for_followees,feature_botfollow,pagerank_follow,pagerank_follower,feature_compute_shortest_path_length,feature_adar
count,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,1.935630e+05,193563.000000,1.935630e+05
mean,-2.349351e-17,-3.524026e-18,3.289091e-17,-6.460714e-18,1.571128e-17,-1.527078e-17,2.055682e-18,-2.936688e-18,-1.119612e-18,-1.732646e-17,2.701753e-17,-1.033347e-17,0.000000,-5.139204e-18
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003,1.000003e+00
min,-2.470019e-01,-4.783731e-01,-3.506885e-01,-1.518837e-01,-8.378495e-02,-3.090646e-01,-5.817618e-02,-2.127364e-01,-5.846780e-02,-4.624011e-01,-2.983737e-01,-3.444038e-01,-2.894660,-7.195994e-02
25%,-2.470019e-01,-4.783731e-01,-3.506885e-01,-1.496996e-01,-8.378495e-02,-3.090646e-01,-5.817618e-02,-2.127364e-01,-5.846780e-02,-3.582375e-01,-2.870929e-01,-3.103661e-01,-0.251711,-7.195994e-02
50%,-2.470019e-01,-4.362154e-01,-3.506885e-01,-1.391148e-01,-8.378495e-02,-2.612517e-01,-5.601421e-02,-2.056222e-01,-5.846780e-02,-2.263797e-01,-2.573236e-01,-2.371852e-01,-0.251711,-7.195994e-02
75%,-1.818570e-01,2.819399e-02,-7.797164e-02,-8.467891e-02,-6.080915e-02,-1.178129e-01,-5.004034e-02,-1.161200e-01,-4.971394e-02,3.586626e-02,-1.508032e-01,-5.046997e-02,0.629272,-5.898665e-02
max,3.245572e+01,1.545724e+01,5.651078e+01,9.911318e+01,2.342005e+02,5.668393e+01,5.002604e+01,7.746680e+01,2.347642e+02,8.050605e+01,1.398284e+01,3.565996e+01,4.153204,2.767408e+02


In [ ]:
# rx = df['feature_preferential']
# x_norm = (rx-np.min(rx))/(np.max(rx)-np.min(rx))
# # #X = df.drop('feature_preferential', axis=1)
# X['feature_preferential'] = x_norm

In [ ]:
X

,feature_jaccard_out_follow,feature_jaccard_out_follower,feature_transitive,feature_preferential,feature_commonfriends_follow,feature_commonfriends_follower,feature_celebrity,feature_cosine_for_followers,feature_cosine_for_followees,feature_botfollow,pagerank_follow,pagerank_follower,feature_compute_shortest_path_length,feature_adar
0,-0.247002,-0.016471,-0.282509,-0.143438,-0.083785,-0.213439,-0.058176,-0.006690,-0.058468,-0.219655,-0.248805,-0.302371,-0.251711,-0.071960
1,-0.247002,0.010449,0.535641,0.075526,-0.083785,0.169065,-0.058176,-0.174476,-0.058468,-0.167383,-0.228583,-0.044492,-0.251711,-0.071960
2,-0.247002,-0.478373,-0.350689,-0.151873,-0.083785,-0.309065,-0.058176,-0.212736,-0.058468,0.368045,-0.274130,-0.343463,-2.894660,-0.071960
3,-0.247002,0.926702,-0.282509,0.248937,-0.083785,1.651265,-0.058176,-0.123736,-0.058468,-0.460936,0.151507,0.017422,-0.251711,-0.071960
4,-0.247002,0.695830,-0.214330,-0.126794,-0.083785,0.025626,-0.058176,0.030012,-0.058468,-0.392406,-0.240343,-0.105066,-0.251711,-0.071960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193558,-0.247002,-0.478373,-0.350689,-0.151817,-0.083785,-0.309065,-0.057924,-0.212736,-0.058468,1.987414,-0.292434,-0.341775,0.629272,-0.071960
193559,-0.247002,-0.387830,-0.350689,-0.098568,-0.083785,-0.213439,0.055585,-0.180098,-0.058468,-0.462401,0.409266,-0.325847,-2.894660,-0.071960
193560,0.318056,-0.175990,0.399283,0.154570,0.008118,0.169065,0.001517,-0.184345,-0.052409,-0.283928,2.256167,-0.259430,-0.251711,-0.015768
193561,-0.200942,-0.478373,-0.282509,-0.143651,-0.072297,-0.309065,-0.055976,-0.212736,-0.030278,0.059593,-0.270282,-0.248639,-0.251711,-0.066071


'/media/kunal/winsec/IISC/ds/HUFFLEPUP/Play/aman'

## LOGISTIC


In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8)
model=linear_model.LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7950559243664919

In [ ]:
weights = model.coef_[0] * (X.max(axis=0) - X.min(axis=0)) / X.std(axis=0)

# Sort the weights in descending order
sorted_idx = weights.argsort()[::-1]
for i in sorted_idx:
    print(f"{feature_list[i]}: {weights[i]}")


feature_adar: 2586.042612506418
feature_cosine_for_followers: 169.49025821290036
feature_commonfriends_follower: 74.21725672515032
feature_transitive: 36.54309578755394
feature_jaccard_out_follower: 12.923535827362796
feature_botfollow: 9.619160688942149
pagerank_follow: 8.745230921953455
feature_jaccard_out_follow: 0.3682229762483209
feature_celebrity: 0.031911695542859574
pagerank_follower: -0.9707918154831172
feature_compute_shortest_path_length: -4.6392259702062395
feature_preferential: -92.8728758872097
feature_cosine_for_followees: -161.71178473719914
feature_commonfriends_follow: -2158.2345324873677


/tmp/ipykernel_8036/1086681370.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{feature_list[i]}: {weights[i]}")


In [ ]:

y_test_predict= model.predict_proba(X_test)
y_test_predict

ypred = []
for i in y_test_predict:
    ypred.append(i[1]);

ypred


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,ypred)

0.5042565080013842

## XGB CLASSIFIER

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7)

# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }



# instantiate the classifier
xgb_clf = XGBClassifier(**params)

# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

In [ ]:
#import xgboost as xgb
#xgb.plot_importance(xgb_clf)

In [ ]:
y_test_predictxgb = xgb_clf.predict_proba(X_test)
y_test_predictxgb_01 = xgb_clf.predict(X_test)
#y_test_predict= model.predict_proba(X_test)
#y_test_predictxgb = xgb_clf.predict_proba(X_test)
ypredxgb = []
for i in y_test_predictxgb:
    ypredxgb.append(i[1]);

ypredxgb


y_test_predictxgb_01



In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,ypredxgb)

0.990498037373801

In [ ]:
from sklearn.metrics import accuracy_score

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_test_predictxgb_01)))

XGBoost model accuracy score: 0.9567


## OUTPUT

In [ ]:
#dout = dout[0:0]
dout = pd.DataFrame(columns=feature_list)
dout

,feature_jaccard_out_follow,feature_jaccard_out_follower,feature_transitive,feature_preferential,feature_commonfriends_follow,feature_commonfriends_follower,feature_celebrity,feature_cosine_for_followers,feature_cosine_for_followees,feature_botfollow,pagerank_follow,pagerank_follower,feature_compute_shortest_path_length,feature_adar


## OUTPUT FOR TEST FILE

In [ ]:
file = open("../test.csv", "r")
dtest = list(csv.reader(file, delimiter=","))
file.close()

dtest = dtest[1:]

from_list = [i[1] for i in dtest]
to_list = [i[2] for i in dtest]

dout = pd.DataFrame(columns=feature_list)
#print(len(from_list))
for i in range(0 , len(from_list)):
    #print(jaccard[i])
   #print(i)
    fn = from_list[i]
    tn = to_list[i]
    #feature_preferential(fn,tn)

    dout.loc[len(dout.index)] = [feature_jaccard_out_follow(fn,tn) ,feature_jaccard_out_follower(fn,tn), feature_transitive(fn,tn),
                            feature_preferential(fn,tn), feature_commonfriends_follow(fn,tn),feature_commonfriends_follower(fn,tn),

                            feature_celebrity(fn,tn),feature_cosine_for_followers(fn,tn),feature_cosine_for_followees(fn,tn)
                             ,feature_botfollow(fn,tn),pr[tn] ,pr[fn],feature_compute_shortest_path_length(fn,tn),feature_adar(fn,tn)
                             ]



    # dout.loc[len(dout.index)] = [feature_jaccard_out(fn,tn) , feature_transitive(fn,tn),
    #                          feature_preferential(fn,tn), feature_commonfriends(fn,tn),
    #                          feature_botfollow(fn,tn),pr[tn],feature_adar(fn,tn)
    #                           ]

   # dout.loc[len(dout.index)] = [fn, tn,feature_jaccard_out(fn,tn)
    #                          ]


In [ ]:
dout[feature_list] = dout[feature_list].astype(float)


In [ ]:
dout['pagerank_follower']

0       2.245257e-07
1       2.828298e-07
2       2.599906e-07
3       2.745523e-07
4       2.556649e-07
            ...     
1995    2.419744e-07
1996    2.299431e-07
1997    2.504190e-07
1998    2.201000e-07
1999    2.072650e-07
Name: pagerank_follower, Length: 2000, dtype: float64

In [ ]:
dout[feature_list] = scaler.transform(dout[feature_list])

In [ ]:
# rx = dout['feature_preferential']
# x_norm = (rx-np.min(rx))/(np.max(rx)-np.min(rx))
# #X = df.drop('feature_preferential', axis=1)
# dout['feature_preferential'] = x_norm

In [ ]:
dout.describe()

,feature_jaccard_out_follow,feature_jaccard_out_follower,feature_transitive,feature_preferential,feature_commonfriends_follow,feature_commonfriends_follower,feature_celebrity,feature_cosine_for_followers,feature_cosine_for_followees,feature_botfollow,pagerank_follow,pagerank_follower,feature_compute_shortest_path_length,feature_adar
count,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2000.000000,2.000000e+03,2.000000e+03,2000.000000,2.000000e+03
mean,4.819042e-03,3.567993e-02,8.104500e+00,1.643471e+04,6.267000e+00,9.218000e+00,1.960736e+00,1.541894e-03,7.335183e-04,14.035256,2.794867e-07,2.593810e-07,2.336000,2.963745e+00
std,1.800970e-02,5.692854e-02,1.891096e+01,6.750473e+04,3.383791e+01,2.297776e+01,2.782771e+01,3.133739e-03,5.174921e-03,25.856256,2.972720e-07,1.478400e-07,0.482929,1.562124e+01
min,7.048052e-17,1.057208e-17,-9.867273e-17,1.200000e+01,-4.713385e-17,4.581234e-17,-6.167045e-18,8.810065e-18,3.358837e-18,0.110116,2.047817e-07,2.051884e-07,2.000000,1.541761e-17
25%,7.048052e-17,1.057208e-17,-9.867273e-17,7.972500e+02,-4.713385e-17,4.581234e-17,-6.167045e-18,8.810065e-18,3.358837e-18,3.866049,2.056416e-07,2.122844e-07,2.000000,1.541761e-17
50%,7.048052e-17,1.331560e-02,2.000000e+00,2.314000e+03,-4.713385e-17,2.000000e+00,-6.167045e-18,5.163217e-04,3.358837e-18,7.754310,2.088800e-07,2.245944e-07,2.000000,1.541761e-17
75%,7.048052e-17,4.732962e-02,8.000000e+00,8.133000e+03,-4.713385e-17,8.000000e+00,-6.167045e-18,1.594896e-03,3.358837e-18,14.715608,2.253129e-07,2.526168e-07,3.000000,1.541761e-17
max,1.944444e-01,5.365854e-01,2.860000e+02,1.650416e+06,6.900000e+02,3.130000e+02,1.011333e+03,3.571429e-02,1.351010e-01,482.000000,5.348346e-06,3.168919e-06,4.000000,2.939334e+02


In [ ]:
Xtest_real=dout[feature_list]
Xtest_real

,feature_jaccard_out_follow,feature_jaccard_out_follower,feature_transitive,feature_preferential,feature_commonfriends_follow,feature_commonfriends_follower,feature_celebrity,feature_cosine_for_followers,feature_cosine_for_followees,feature_botfollow,pagerank_follow,pagerank_follower,feature_compute_shortest_path_length,feature_adar
0,3.325527e-02,8.372093e-02,3.200000e+01,13020.0,7.100000e+01,1.800000e+01,3.842975e-01,1.382488e-03,5.453149e-03,14.028571,2.513444e-07,2.245257e-07,2.0,5.003642e+01
1,7.048052e-17,1.057208e-17,-9.867273e-17,1242.0,-4.713385e-17,4.581234e-17,-6.167045e-18,8.810065e-18,3.358837e-18,1.782609,2.060506e-07,2.828298e-07,3.0,1.541761e-17
2,7.048052e-17,1.057208e-17,-9.867273e-17,7440.0,-4.713385e-17,4.581234e-17,-6.167045e-18,8.810065e-18,3.358837e-18,28.209677,2.062591e-07,2.599906e-07,3.0,1.541761e-17
3,7.048052e-17,1.779359e-02,3.000000e+00,13888.0,-4.713385e-17,5.000000e+00,-6.167045e-18,3.600230e-04,3.358837e-18,1.290179,2.210491e-07,2.745523e-07,2.0,1.541761e-17
4,7.048052e-17,1.180328e-01,3.000000e+01,28980.0,-4.713385e-17,3.600000e+01,-6.167045e-18,1.242236e-03,3.358837e-18,8.130435,2.283699e-07,2.556649e-07,2.0,1.541761e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,7.048052e-17,2.328590e-02,6.000000e+00,79414.0,-4.713385e-17,1.800000e+01,-6.167045e-18,2.266603e-04,3.358837e-18,3.805085,5.593386e-07,2.419744e-07,2.0,1.541761e-17
1996,3.547297e-03,1.315789e-02,5.000000e+00,35164.0,2.100000e+01,5.000000e+00,1.183551e-01,1.421909e-04,5.972017e-04,26.489933,2.626940e-07,2.299431e-07,2.0,7.456276e+00
1997,7.048052e-17,8.928571e-02,4.000000e+00,550.0,-4.713385e-17,5.000000e+00,-6.167045e-18,9.090909e-03,3.358837e-18,2.620000,2.167688e-07,2.504190e-07,2.0,1.541761e-17
1998,7.048052e-17,1.057208e-17,1.000000e+00,110.0,-4.713385e-17,4.581234e-17,-6.167045e-18,8.810065e-18,3.358837e-18,10.700000,2.047997e-07,2.201000e-07,2.0,1.541761e-17


## LOGISTIC REGRESSION on test

In [ ]:
y_predicted_log = model.predict_proba(Xtest_real)

dout.to_csv('logistic_testingbiohazard.csv', index=False)


In [ ]:
result = []

with open("output/logistic_10lakh.csv" , 'w' , newline="") as f:
    writer = csv.writer(f)

    for i in range(0, len(from_list)):
        va= format(y_predicted_log[i][1] , ".2f")
        writer.writerow([ i+1 , va ])


## XGBOOST final on test

In [ ]:

y_predicted_xgb = xgb_clf.predict_proba(Xtest_real)
dout.to_csv('xgb_testingbiohazard.csv', index=False)

In [ ]:
result = []

with open("output/xgb_10lakh.csv" , 'w' , newline="") as f:
    writer = csv.writer(f)

    for i in range(0, len(from_list)):
        va= format(y_predicted_xgb[i][1] , ".2f")
        writer.writerow([ i+1 , va ])

